In [ ]:
# ONS population estimates for Lewisham.
# Fetches ONS data and prepares it for use in Lewisham ward profiles.
#
# This requires the OA->LSOA lookup table produced by "lbl_boundaries_oa.ipynb"
# as well as the OA->WD22 lookup table produced by "lbl_boundaries_2022_wards.ipynb"

In [1]:
import numpy as np
import pandas as pd

from google.colab import drive
import google.colab.files as files

import IPython

Tools
==
Helpers to reformat & aggregate the source data.

In [2]:
# Aggregate OA-level data to LSOAs, Wards, etc. (Calculates the sum.)
def aggregate_oa_groups(oa_data, oa_group_table, oa_col, group_col):
  return pd.merge(oa_group_table, oa_data, on=oa_col, how='right').\
              drop(columns=[oa_col]).\
              groupby(group_col).sum()

Data
==
GDrive mount
--

In [3]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
# Used as root folder.
project_dir = '/content/gdrive/MyDrive/WardProfiles'
oa_lookups_dir = f"{project_dir}/lookups/oa"
ward_lookups_dir = f"{project_dir}/lookups/2022_wards"

# For exports
pop_dir = f"{project_dir}/population"

In [ ]:
!mkdir -p '{pop_dir}'

Downloads
--

In [ ]:
# Pop est at OA level in London, mid-2020
# Source: 
# https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates/datasets/censusoutputareaestimatesinthelondonregionofengland
!wget 'https://www.ons.gov.uk/file?uri=%2fpeoplepopulationandcommunity%2fpopulationandmigration%2fpopulationestimates%2fdatasets%2fcensusoutputareaestimatesinthelondonregionofengland%2fmid2020sape23dt10a/sape23dt10amid2020coaunformattedsyoaestimateslondon.xlsx' \
  -O "{pop_dir}/sape23dt10amid2020coaunformattedsyoaestimateslondon.xlsx"

--2021-11-02 15:47:25--  https://www.ons.gov.uk/file?uri=%2fpeoplepopulationandcommunity%2fpopulationandmigration%2fpopulationestimates%2fdatasets%2fcensusoutputareaestimatesinthelondonregionofengland%2fmid2020sape23dt10a/sape23dt10amid2020coaunformattedsyoaestimateslondon.xlsx
Resolving www.ons.gov.uk (www.ons.gov.uk)... 104.16.230.22, 104.16.231.22, 2606:4700::6810:e716, ...
Connecting to www.ons.gov.uk (www.ons.gov.uk)|104.16.230.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘/content/gdrive/MyDrive/WardProfiles/population/sape23dt10amid2020coaunformattedsyoaestimateslondon.xlsx’

/content/gdrive/MyD     [           <=>      ]  27.43M  8.73MB/s    in 3.1s    

2021-11-02 15:47:31 (8.73 MB/s) - ‘/content/gdrive/MyDrive/WardProfiles/population/sape23dt10amid2020coaunformattedsyoaestimateslondon.xlsx’ saved [28762871]



In [ ]:
# U.K. pop est at misc administrative levels, mid-2020
# Source: 
# https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates/datasets/populationestimatesforukenglandandwalesscotlandandnorthernireland
!wget 'https://www.ons.gov.uk/file?uri=%2fpeoplepopulationandcommunity%2fpopulationandmigration%2fpopulationestimates%2fdatasets%2fpopulationestimatesforukenglandandwalesscotlandandnorthernireland%2fmid2020/ukpopestimatesmid2020on2021geography.xls' \
  -O "{pop_dir}/ukpopestimatesmid2020on2021geography.xlsx"

--2021-11-16 10:56:21--  https://www.ons.gov.uk/file?uri=%2fpeoplepopulationandcommunity%2fpopulationandmigration%2fpopulationestimates%2fdatasets%2fpopulationestimatesforukenglandandwalesscotlandandnorthernireland%2fmid2020/ukpopestimatesmid2020on2021geography.xls
Resolving www.ons.gov.uk (www.ons.gov.uk)... 104.16.230.22, 104.16.231.22, 2606:4700::6810:e616, ...
Connecting to www.ons.gov.uk (www.ons.gov.uk)|104.16.230.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/vnd.ms-excel]
Saving to: ‘/content/gdrive/MyDrive/WardProfiles/population/ukpopestimatesmid2020on2021geography.xlsx’

/content/gdrive/MyD     [   <=>              ]   2.46M  5.48MB/s    in 0.4s    

2021-11-16 10:56:22 (5.48 MB/s) - ‘/content/gdrive/MyDrive/WardProfiles/population/ukpopestimatesmid2020on2021geography.xlsx’ saved [2576896]



Lookups
--
Used to match up OAs with their LSOAs, and 2022 electoral wards

In [5]:
# All OAs and their LSOAs in Lewisham
oa_lsoa_join = pd.read_csv(f"{oa_lookups_dir}/lbl_oa11_lsoa11_msoa11_lad20_rgn20_202012.csv")
oa_lsoa_join = oa_lsoa_join[['OA11CD', 'LSOA11CD']]
oa_lsoa_join.head()

,OA11CD,LSOA11CD
0,E00016277,E01003220
1,E00016278,E01003220
2,E00016285,E01003220
3,E00016257,E01003221
4,E00016263,E01003221


In [6]:
# All OAs and their 2022 wards in Lewisham
oa_wd22_join = pd.read_csv(f"{ward_lookups_dir}/lbl_oa11_wd22_proposed.csv")
oa_wd22_join = oa_wd22_join[['OA11CD', 'WD22CD_proposed']]
oa_wd22_join.head()

,OA11CD,WD22CD_proposed
0,E00016403,E05013721
1,E00016442,E05013721
2,E00016407,E05013721
3,E00016404,E05013721
4,E00016402,E05013721


Process & export
==

Lewisham - single-year age totals
--

In [8]:
# For filtering of source data
lbl_oa_list = oa_lsoa_join.OA11CD.unique() # All OAs in Lewisham
len(lbl_oa_list)

887

In [9]:
# Raw single-year plus totals
for sheetname, datname in zip(
    ['Mid-2020 Persons', 'Mid-2020 Males', 'Mid-2020 Females'],
    ['pop_est_2020_all', 'pop_est_2020_male', 'pop_est_2020_female']):

  IPython.display.display(f"== {sheetname} ==")

  d = pd.read_excel(f"{pop_dir}/sape23dt10amid2020coaunformattedsyoaestimateslondon.xlsx", 
                    sheet_name=sheetname,
                    header=4, # 5th row has column names
  ) 
  IPython.display.display(len(d))
  IPython.display.display(d.head())

  # Remove spaces in oclumn names
  d = d.rename(columns={'All Ages': 'Total'})

  # Filter & transform to OA index data
  lbl_oa_age = d[d.OA11CD.isin(lbl_oa_list)].drop(columns=['LSOA11CD'])
  lbl_oa_age.to_csv(f"{pop_dir}/lbl_{datname}_oa11.csv", index=False)
  IPython.display.display(len(lbl_oa_age))
  IPython.display.display(lbl_oa_age.head())

  # Aggregate to LSOA level
  lbl_lsoa_age = aggregate_oa_groups(lbl_oa_age, oa_lsoa_join, oa_col='OA11CD', group_col='LSOA11CD')
  lbl_lsoa_age.to_csv(f"{pop_dir}/lbl_{datname}_lsoa11.csv", index=True)
  IPython.display.display(len(lbl_lsoa_age))
  IPython.display.display(lbl_lsoa_age.head())

  # Aggregate to WD22 level
  lbl_wd22_age = aggregate_oa_groups(lbl_oa_age, oa_wd22_join, oa_col='OA11CD', group_col='WD22CD_proposed')
  lbl_wd22_age.to_csv(f"{pop_dir}/lbl_{datname}_wd22.csv", index=True)
  IPython.display.display(len(lbl_wd22_age))
  IPython.display.display(lbl_wd22_age.head())

'== Mid-2020 Persons =='

25053

,OA11CD,LSOA11CD,All Ages,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,...,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90+
0,E00000001,E01000001,250,0,0,2,0,4,7,10,0,3,11,4,1,5,2,2,2,0,3,0,1,4,1,2,1,1,0,5,6,2,0,1,3,1,2,0,2,2,...,7,10,2,3,2,2,3,3,6,5,5,1,1,5,7,3,7,0,7,2,4,6,1,4,2,0,2,3,2,5,0,3,4,2,0,0,1,0,0,7
1,E00000003,E01000001,323,0,2,3,0,2,5,5,4,7,10,13,2,7,5,1,3,3,5,1,0,2,3,2,3,2,6,2,0,4,1,2,3,3,2,3,1,3,...,9,7,1,2,3,2,7,3,5,2,4,7,2,7,5,9,13,7,4,7,7,13,5,4,1,2,0,0,2,2,2,0,0,0,2,0,0,0,0,11
2,E00000005,E01000001,143,0,0,2,2,2,0,3,0,3,3,1,1,0,2,0,1,2,0,0,0,0,0,1,4,1,1,0,0,1,2,1,1,0,1,3,0,0,...,4,3,2,3,2,4,1,1,3,9,1,1,3,1,4,1,2,5,0,1,2,0,6,3,1,0,1,1,0,1,1,0,0,0,0,0,0,1,0,4
3,E00000007,E01000001,169,3,1,2,4,2,0,2,0,3,0,0,5,0,5,0,1,0,0,1,1,0,0,4,2,6,4,8,9,8,1,3,3,2,2,8,4,3,...,1,0,4,4,2,0,2,0,2,3,1,3,1,1,0,0,3,3,2,2,0,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,E00166756,E01000001,240,4,1,1,3,4,0,3,0,3,3,2,0,1,0,0,2,0,0,0,1,0,2,2,8,2,7,7,7,8,13,5,9,4,3,4,5,3,...,3,4,4,6,2,3,2,0,3,1,1,1,1,1,0,2,3,1,0,4,1,3,3,0,3,0,3,1,4,1,2,1,0,0,3,1,2,0,1,0


887

,OA11CD,Total,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,...,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90+
16070,E00016115,326,1,2,2,3,4,3,0,4,9,4,8,2,6,4,7,7,6,2,1,5,9,4,3,1,5,5,2,3,3,3,3,3,2,3,3,3,4,4,...,7,5,9,3,4,9,6,6,1,6,10,3,2,5,5,4,4,0,2,2,4,7,5,2,4,3,1,1,1,4,0,0,0,0,1,1,1,0,1,1
16071,E00016136,314,4,3,1,5,5,5,2,5,4,4,6,6,8,9,7,6,4,3,2,5,1,3,5,3,5,2,3,3,2,1,2,4,1,2,3,10,5,6,...,6,4,11,5,4,4,5,2,6,3,5,2,1,3,5,5,1,2,1,2,2,2,2,3,0,3,0,0,0,0,0,0,1,3,0,1,0,0,4,2
16072,E00016137,404,4,8,4,9,5,9,15,6,11,8,6,6,9,8,13,8,7,3,6,4,5,6,4,5,7,3,4,4,3,3,4,6,4,4,5,6,6,4,...,4,7,6,5,5,8,1,6,5,2,5,7,4,1,2,3,1,2,2,1,1,1,2,0,2,2,2,3,2,1,1,0,1,0,3,0,0,1,0,0
16073,E00016138,295,2,7,4,6,6,7,3,6,2,7,7,4,4,5,5,6,4,4,6,4,3,3,4,4,2,3,2,1,2,2,3,4,5,7,8,4,5,6,...,5,2,5,4,3,5,7,4,5,3,3,2,1,0,2,2,2,1,2,1,4,1,1,1,0,2,0,1,0,1,0,0,0,1,0,0,0,0,1,3
16074,E00016139,352,5,7,1,9,2,9,7,5,11,8,3,8,10,8,4,7,5,7,5,4,4,4,1,5,5,4,4,4,2,3,4,2,5,5,4,4,6,6,...,7,5,5,9,2,1,8,4,4,7,4,3,4,2,6,1,0,2,1,0,1,2,3,5,0,0,1,2,0,0,2,2,0,0,0,0,0,0,0,0


...

,OA11CD,Total,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,...,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90+
16070,E00016115,162,1,2,0,1,1,0,0,2,4,2,5,2,3,3,3,6,4,1,1,3,2,2,2,0,2,3,0,1,2,1,1,2,0,2,1,1,2,3,...,3,2,5,2,3,4,3,3,0,3,5,2,1,2,2,3,2,0,0,1,3,4,1,1,0,2,0,1,0,2,0,0,0,0,1,1,1,0,1,1
16071,E00016136,154,2,2,0,3,3,2,0,1,4,2,6,2,4,5,4,1,2,2,1,2,0,0,2,1,2,1,2,2,1,0,1,3,0,1,1,6,0,2,...,3,4,9,2,2,3,2,1,3,2,1,0,1,1,4,3,0,2,0,2,1,1,2,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,3,2
16072,E00016137,211,1,4,3,3,3,6,9,6,6,4,3,1,3,4,2,4,3,1,0,3,3,3,1,1,3,3,1,2,3,2,2,5,3,2,3,5,3,2,...,1,3,3,3,4,4,1,2,1,2,2,4,1,0,2,1,1,2,2,0,1,1,1,0,2,2,0,2,0,1,1,0,0,0,3,0,0,1,0,0
16073,E00016138,161,1,3,3,5,1,0,1,3,1,3,4,3,1,2,3,0,2,2,5,0,2,3,2,2,1,1,0,0,2,1,2,4,4,5,4,3,2,3,...,2,2,3,1,2,4,5,3,3,1,2,2,0,0,2,0,2,1,0,0,3,1,0,1,0,2,0,1,0,1,0,0,0,1,0,0,0,0,1,3
16074,E00016139,167,1,5,0,3,1,5,4,0,4,5,1,2,3,3,1,2,2,4,1,1,2,2,0,2,4,3,4,2,1,2,2,1,3,3,3,1,2,3,...,3,3,2,6,0,1,5,1,3,4,3,1,4,0,3,0,0,0,0,0,1,0,2,2,0,0,0,2,0,0,1,1,0,0,0,0,0,0,0,0


169

,Total,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90+
LSOA11CD,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
E01003189,855,6,16,6,15,9,13,14,12,19,16,19,10,14,17,13,13,13,10,8,9,9,10,7,6,12,11,7,7,9,6,8,15,10,13,12,16,9,13,10,...,12,14,22,14,11,16,16,10,10,12,13,9,7,3,13,7,5,5,2,3,9,7,6,6,2,6,0,6,0,4,2,1,0,1,4,2,1,1,5,6
E01003190,1264,29,22,29,27,21,19,22,17,13,17,25,15,19,18,14,9,16,11,13,12,15,11,15,12,15,26,15,15,21,15,28,33,37,31,32,19,19,15,15,...,18,18,9,17,12,15,17,15,12,12,15,7,11,4,8,7,5,0,3,0,2,6,4,4,6,9,3,3,3,1,4,1,2,0,0,1,1,0,1,4
E01003191,875,9,10,9,12,20,28,9,24,29,24,4,7,10,12,10,9,4,4,5,6,3,6,10,10,9,10,12,20,9,12,12,17,23,22,18,13,15,15,15,...,16,10,5,9,20,20,13,14,9,10,9,17,8,8,4,5,4,4,4,3,4,2,4,3,8,2,1,1,0,0,2,1,0,1,5,4,2,0,3,5
E01003192,903,10,13,16,14,19,12,16,5,21,11,12,12,9,13,13,6,12,12,7,6,8,10,8,15,5,13,10,24,13,22,10,17,14,16,13,17,15,13,16,...,13,8,10,9,9,11,11,5,8,8,10,7,7,8,5,8,6,1,8,11,9,6,5,0,1,2,3,1,2,8,6,6,2,4,2,1,0,1,1,8
E01003193,867,7,13,13,13,12,11,12,14,7,13,10,11,10,7,13,8,6,12,6,9,7,6,5,12,9,14,9,13,14,10,12,12,14,16,20,11,11,13,13,...,12,11,4,18,10,10,11,12,3,17,10,5,5,8,6,4,4,6,6,12,10,15,12,11,7,3,8,5,6,4,8,2,3,0,2,2,1,1,0,6


19

,Total,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90+
WD22CD_proposed,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
E05013714,6014,73,82,80,81,93,100,84,90,124,102,99,75,84,96,78,72,67,64,56,50,43,55,54,60,61,76,64,83,68,74,77,108,112,112,102,99,85,96,93,...,99,85,81,93,78,105,92,72,67,73,78,64,58,52,52,49,39,25,28,33,35,35,31,19,24,24,19,22,10,20,27,22,18,12,12,14,13,4,10,37
E05013715,8658,117,118,117,118,116,110,98,118,112,100,94,77,69,91,75,79,77,73,78,64,54,64,78,87,94,157,145,181,200,202,186,217,197,176,175,183,150,152,166,...,138,98,114,103,105,100,100,88,76,90,71,63,83,66,65,57,45,55,52,47,54,64,52,54,42,35,44,54,27,33,27,33,32,36,27,24,23,17,19,92
E05013716,9508,125,116,112,106,103,87,101,113,84,85,68,66,62,83,62,71,74,58,76,131,207,218,247,239,270,241,245,280,302,280,258,271,207,219,230,178,197,198,167,...,101,109,97,94,75,85,90,67,77,72,65,61,67,42,55,48,43,39,36,38,42,38,38,29,27,24,27,20,19,21,22,22,12,13,13,5,13,11,7,39
E05013717,8954,100,94,102,110,109,130,128,143,137,118,112,108,121,104,112,104,88,116,105,76,66,81,92,99,91,120,94,105,106,111,112,141,123,130,116,121,139,138,135,...,125,135,128,118,116,164,152,151,115,116,104,97,93,74,89,90,73,70,79,60,67,69,66,45,32,49,46,36,38,30,35,42,56,32,35,39,21,24,18,98
E05013718,7971,134,134,126,127,128,132,128,111,141,89,95,96,72,74,87,52,57,48,49,49,39,37,66,65,65,109,128,114,136,136,150,162,152,159,179,172,162,190,178,...,108,88,88,90,92,91,97,102,81,79,78,80,71,67,46,72,56,45,53,42,37,44,42,32,45,36,23,34,29,28,21,18,23,15,16,20,15,13,6,34


Reference (England, London) - single-year age groups
--

In [15]:
# Preferred spelling/captialisation for reference geographies in the source data.
# (Only for reference geographies we want to include in our data.)
reference_names = {
                   'UNITED KINGDOM': 'United Kingdom', 
                   'GREAT BRITAIN': 'Great Britain', 
                   'ENGLAND AND WALES': 'England and Wales', 
                   'ENGLAND': 'England', 
                   'LONDON': 'London', 
                   'Lewisham': 'Lewisham'}

In [17]:
# Raw single-year plus totals
for sheetname, datname in zip(
    ['MYE2 - Persons', 'MYE2 - Males', 'MYE2 - Females'],
    ['pop_est_2020_all', 'pop_est_2020_male', 'pop_est_2020_female']):

  IPython.display.display(f"== {sheetname} ==")

  d = pd.read_excel(f"{pop_dir}/ukpopestimatesmid2020on2021geography.xlsx", 
                    sheet_name=sheetname,
                    header=7, # 8th row has column names
  ) 
  IPython.display.display(len(d))
  IPython.display.display(d.head())

  # Remove spaces in column names
  d = d.rename(columns={'All ages': 'Total'})

  # Filter geographies & map to preferred spelling
  d = d[d.Name.isin(reference_names.keys())]
  d.Name = d.Name.map(reference_names)

  # Export
  d.to_csv(f"{pop_dir}/references_{datname}.csv", index=False)
  IPython.display.display(len(d))
  IPython.display.display(d.head())

'== MYE2 - Persons =='

420

,Code,Name,Geography,All ages,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,...,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90+
0,K02000001,UNITED KINGDOM,Country,67081234,701897,730219,759354,783321,807539,806745,813073,830700,855000,841895,826080,817287,823732,796700,781315,752712,740693,722928,717252,750095,778930,810303,827261,856602,860062,858738,884489,887131,914604,931668,914186,905920,913691,891015,897163,894424,...,923655,923357,934714,932611,938738,928163,906643,884567,852740,816209,796532,777771,747241,718065,689198,687334,674764,651687,652398,660817,672642,702415,753009,575023,549939,540477,494925,435050,383368,386890,373288,350944,322102,292999,264997,231822,206428,185189,161430,609503
1,K03000001,GREAT BRITAIN,Country,65185724,679949,707375,735831,759033,782406,781788,788058,805285,828652,815756,800239,791126,797515,771738,757479,729523,717775,700554,694916,729044,757338,787678,804761,833762,837295,835880,861037,863596,889805,906493,889121,880885,888368,865467,871552,869245,...,897689,896933,908493,906366,912069,902013,881162,859731,828373,792963,774040,755706,726142,697947,670112,668622,656553,634282,635119,643686,655795,685799,736575,559989,535276,526259,481514,423192,372984,377171,363738,342324,313997,285821,258489,226198,201222,180762,157634,595576
2,K04000001,ENGLAND AND WALES,Country,59719724,631314,656313,682886,704584,725691,724207,730120,746114,767986,753209,740601,729825,736497,713101,699992,672530,661885,646305,640353,668619,693691,721232,736190,762062,765904,764774,788010,788741,810749,827333,812854,805114,812584,792059,798713,797036,...,819346,817221,828361,826817,829962,820880,800342,781015,751805,719099,701008,684916,657520,631686,607106,606768,596365,576571,577445,586144,597525,626322,673067,512956,491503,482469,440549,386545,339949,344349,332721,313223,287709,261753,236742,207218,184755,165926,145102,551827
3,E92000001,ENGLAND,Country,56550138,601913,625476,650226,671016,690816,689190,694734,709940,730548,715046,703087,692873,698821,676773,664025,637756,628023,613025,606611,630456,653155,679487,694216,720400,725264,725111,745909,747190,768513,784770,771964,764738,773176,753953,760821,758955,...,775715,774083,784116,782532,784299,775320,756023,736939,709201,677962,660419,644896,618893,594643,571143,569908,559844,541297,542108,550059,560762,587908,632852,482547,461855,453442,413706,362412,318577,323778,313081,294838,271063,246429,223058,195010,173861,156497,136917,521067
4,E12000001,NORTH EAST,Region,2680763,25311,26882,27877,28895,29813,29512,30386,31170,32135,32174,31686,30906,31329,30258,30424,28652,28362,27866,28328,33383,35284,36140,35693,34704,34348,34051,35187,35692,37513,37376,35409,34327,34349,33447,34356,33596,...,35714,36388,37571,38163,39187,38652,38852,38314,36658,35494,35760,35487,33725,32222,30802,30557,30112,28805,28903,29297,29715,30809,33023,24663,23451,22422,19734,17906,16386,16625,16170,15310,14109,12247,11364,9734,8646,7607,6658,22576


6

,Code,Name,Geography,Total,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,...,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90+
0,K02000001,United Kingdom,Country,67081234,701897,730219,759354,783321,807539,806745,813073,830700,855000,841895,826080,817287,823732,796700,781315,752712,740693,722928,717252,750095,778930,810303,827261,856602,860062,858738,884489,887131,914604,931668,914186,905920,913691,891015,897163,894424,...,923655,923357,934714,932611,938738,928163,906643,884567,852740,816209,796532,777771,747241,718065,689198,687334,674764,651687,652398,660817,672642,702415,753009,575023,549939,540477,494925,435050,383368,386890,373288,350944,322102,292999,264997,231822,206428,185189,161430,609503
1,K03000001,Great Britain,Country,65185724,679949,707375,735831,759033,782406,781788,788058,805285,828652,815756,800239,791126,797515,771738,757479,729523,717775,700554,694916,729044,757338,787678,804761,833762,837295,835880,861037,863596,889805,906493,889121,880885,888368,865467,871552,869245,...,897689,896933,908493,906366,912069,902013,881162,859731,828373,792963,774040,755706,726142,697947,670112,668622,656553,634282,635119,643686,655795,685799,736575,559989,535276,526259,481514,423192,372984,377171,363738,342324,313997,285821,258489,226198,201222,180762,157634,595576
2,K04000001,England and Wales,Country,59719724,631314,656313,682886,704584,725691,724207,730120,746114,767986,753209,740601,729825,736497,713101,699992,672530,661885,646305,640353,668619,693691,721232,736190,762062,765904,764774,788010,788741,810749,827333,812854,805114,812584,792059,798713,797036,...,819346,817221,828361,826817,829962,820880,800342,781015,751805,719099,701008,684916,657520,631686,607106,606768,596365,576571,577445,586144,597525,626322,673067,512956,491503,482469,440549,386545,339949,344349,332721,313223,287709,261753,236742,207218,184755,165926,145102,551827
3,E92000001,England,Country,56550138,601913,625476,650226,671016,690816,689190,694734,709940,730548,715046,703087,692873,698821,676773,664025,637756,628023,613025,606611,630456,653155,679487,694216,720400,725264,725111,745909,747190,768513,784770,771964,764738,773176,753953,760821,758955,...,775715,774083,784116,782532,784299,775320,756023,736939,709201,677962,660419,644896,618893,594643,571143,569908,559844,541297,542108,550059,560762,587908,632852,482547,461855,453442,413706,362412,318577,323778,313081,294838,271063,246429,223058,195010,173861,156497,136917,521067
213,E12000007,London,Region,9002488,115245,116979,119607,121178,122790,120988,120075,122677,125200,117393,114196,111526,111947,108425,104659,100322,97634,96754,92855,86891,88563,95773,108966,125734,137558,143752,148014,150226,155293,160563,161864,164327,167537,163383,164973,162737,...,115029,114370,112893,111724,108924,107054,102375,97182,93187,89647,84731,80776,76828,73594,69470,66040,63250,60684,58698,58124,57109,57299,59413,48487,44594,43019,40209,36190,32407,33782,32593,30682,27909,26014,23600,20651,18566,16592,14399,58672


...

'== MYE2 - Females =='

420

,Code,Name,Geography,All ages,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,...,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90+
0,K02000001,UNITED KINGDOM,Country,33935525,341795,354887,369889,381457,392912,392808,396258,404941,416995,410453,403269,398348,401397,387546,381039,366608,360365,350993,347648,364798,376139,392006,400414,416001,416566,418173,431224,435214,453049,458161,449797,450682,457966,450660,449359,449233,...,470032,469525,473125,474724,477697,472307,461124,449311,433795,415958,406314,396633,381506,366881,353453,354014,347575,337435,338485,344326,349927,366752,393528,302509,291001,286767,265624,235783,210096,212739,206959,196217,182702,168423,154056,137230,124028,113621,100919,412286
1,K03000001,GREAT BRITAIN,Country,32974170,331201,343797,358402,369649,380824,380641,384059,392457,404158,397585,390606,385622,388606,375453,369442,355370,349266,340036,336732,354844,366135,381176,389663,404839,405563,407097,419634,423475,440659,445514,437273,438148,445202,437781,436361,436490,...,456675,456079,459681,461390,464050,458978,448329,436589,421380,404049,394858,385385,370717,356684,343882,344682,338297,328586,329684,335495,341194,358032,384788,294511,283189,279186,258373,229369,204342,207328,201609,191344,178018,164180,150162,133824,120808,110820,98499,402455
2,K04000001,ENGLAND AND WALES,Country,30173382,307514,319044,332653,343350,353465,352594,355901,363588,374713,367048,361287,355554,358653,346775,341254,327703,322167,313556,310042,324867,334829,348432,355765,369410,370487,372071,383451,385816,400919,406057,399137,400073,406975,400470,399678,399676,...,415958,414663,418498,420210,421717,416832,406579,396216,381986,366126,357059,348755,335386,322499,311180,312728,307096,298611,299612,305570,310759,326913,351223,269635,259589,255631,235837,208817,185581,188688,183889,174604,162505,149767,137094,122113,110645,101443,90483,372365
3,E92000001,ENGLAND,Country,28567320,293098,304098,316666,326927,336364,335535,338730,345954,356419,348459,342943,337660,340266,329091,323745,310632,305653,297528,293850,306756,315496,328567,335796,349599,351363,353228,363407,365436,379850,385156,379113,379925,387224,381253,380725,380382,...,393563,392445,395838,397436,398256,393381,383788,373754,360010,344951,336192,328213,315661,303307,292765,293735,288334,280496,281256,287053,291941,307142,330413,253652,244025,240429,221693,195845,173926,177589,173173,164396,153030,140983,129198,114852,104116,95623,85372,351519
4,E12000001,NORTH EAST,Region,1363014,12299,13094,13580,14229,14402,14183,14687,15138,15616,15746,15444,14992,15281,14706,14911,13957,13759,13380,13639,16239,16986,17351,17115,16801,16573,16248,17013,17499,18692,18375,17398,17110,17477,17053,17416,17218,...,18418,18739,19267,19561,20047,19897,20019,19610,18871,18251,18340,18286,17157,16569,15842,15680,15502,14887,14984,15396,15378,16100,17112,12926,12284,11786,10522,9655,8888,9298,9019,8690,8121,7120,6685,5809,5265,4662,4220,15319


6

,Code,Name,Geography,Total,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,...,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90+
0,K02000001,United Kingdom,Country,33935525,341795,354887,369889,381457,392912,392808,396258,404941,416995,410453,403269,398348,401397,387546,381039,366608,360365,350993,347648,364798,376139,392006,400414,416001,416566,418173,431224,435214,453049,458161,449797,450682,457966,450660,449359,449233,...,470032,469525,473125,474724,477697,472307,461124,449311,433795,415958,406314,396633,381506,366881,353453,354014,347575,337435,338485,344326,349927,366752,393528,302509,291001,286767,265624,235783,210096,212739,206959,196217,182702,168423,154056,137230,124028,113621,100919,412286
1,K03000001,Great Britain,Country,32974170,331201,343797,358402,369649,380824,380641,384059,392457,404158,397585,390606,385622,388606,375453,369442,355370,349266,340036,336732,354844,366135,381176,389663,404839,405563,407097,419634,423475,440659,445514,437273,438148,445202,437781,436361,436490,...,456675,456079,459681,461390,464050,458978,448329,436589,421380,404049,394858,385385,370717,356684,343882,344682,338297,328586,329684,335495,341194,358032,384788,294511,283189,279186,258373,229369,204342,207328,201609,191344,178018,164180,150162,133824,120808,110820,98499,402455
2,K04000001,England and Wales,Country,30173382,307514,319044,332653,343350,353465,352594,355901,363588,374713,367048,361287,355554,358653,346775,341254,327703,322167,313556,310042,324867,334829,348432,355765,369410,370487,372071,383451,385816,400919,406057,399137,400073,406975,400470,399678,399676,...,415958,414663,418498,420210,421717,416832,406579,396216,381986,366126,357059,348755,335386,322499,311180,312728,307096,298611,299612,305570,310759,326913,351223,269635,259589,255631,235837,208817,185581,188688,183889,174604,162505,149767,137094,122113,110645,101443,90483,372365
3,E92000001,England,Country,28567320,293098,304098,316666,326927,336364,335535,338730,345954,356419,348459,342943,337660,340266,329091,323745,310632,305653,297528,293850,306756,315496,328567,335796,349599,351363,353228,363407,365436,379850,385156,379113,379925,387224,381253,380725,380382,...,393563,392445,395838,397436,398256,393381,383788,373754,360010,344951,336192,328213,315661,303307,292765,293735,288334,280496,281256,287053,291941,307142,330413,253652,244025,240429,221693,195845,173926,177589,173173,164396,153030,140983,129198,114852,104116,95623,85372,351519
213,E12000007,London,Region,4488110,56125,57008,58106,59393,59752,58965,58313,60099,61102,57071,55425,54568,54555,52486,51010,48584,47562,47354,44670,41455,42088,45483,53627,62608,68718,72364,73157,74879,76594,78905,78329,79860,81388,79634,78855,77316,...,58287,57534,56751,56827,55436,54339,52462,49883,47557,45975,43235,41293,39406,37733,35832,34763,33288,32138,31060,30966,30627,30649,31550,26416,24385,23523,22361,20266,18191,18689,18300,17399,15994,14960,13785,12028,11087,9983,8870,38201


Lewisham 5-year age groups
--

In [18]:
# Expects single-age columns, as provided by the ONS estimates.
# Aggregates into binned age groups.
# This is not super modular or reusable -- see next section for an alternative approach.
def aggregate_age_bins(d, fixed_columns, age_columns, bin_size, max_bin):
  group_splits = range(bin_size, max_bin+1, bin_size)
  groups = np.split(age_columns, group_splits)
  out = d[fixed_columns].copy()
  for group in groups:
    name = "%s-%s" % (min(group), max(group))
    out[name] = d[group].sum(axis=1)
  # Adjust the name of the last group to be open ended
  out = out.rename(columns={out.columns[-1]: '%s+' % max_bin})
  return out

In [19]:
# Aggregate into 5-year bins
for datname_src, datname_dst in zip(
    ['pop_est_2020_all', 'pop_est_2020_male', 'pop_est_2020_female'],
    ['pop_est_2020_5ybins_all', 'pop_est_2020_5ybins_male', 'pop_est_2020_5ybins_female']):

  IPython.display.display(f"== {datname_src} ==")

  for level in ['oa11', 'lsoa11', 'wd22']:
    IPython.display.display(f"=== {level} ===")
    d = pd.read_csv(f"{pop_dir}/lbl_{datname_src}_{level}.csv")
    out = aggregate_age_bins(d,
                             fixed_columns=[d.columns[0], 'Total'],
                             age_columns=[str(a) for a in range(90)] + ['90+'],
                             bin_size=5,
                             max_bin=85)
    out.to_csv(f"{pop_dir}/lbl_{datname_dst}_{level}.csv", index=False)
    IPython.display.display(out.head())

'== pop_est_2020_all =='

'=== oa11 ==='

,OA11CD,Total,0-4,5-9,10-14,15-19,20-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80-84,85+
0,E00016115,326,12,20,27,21,22,16,14,18,21,20,29,26,26,15,20,10,4,5
1,E00016136,314,18,20,36,20,17,11,12,31,27,20,28,21,14,14,11,3,4,7
2,E00016137,404,30,49,42,28,27,17,23,24,31,26,30,25,19,10,5,11,3,4
3,E00016138,295,25,25,25,24,16,10,27,20,24,19,21,24,9,9,8,3,2,4
4,E00016139,352,24,40,33,28,19,17,20,23,30,17,34,19,20,10,11,3,4,0


'=== lsoa11 ==='

,LSOA11CD,Total,0-4,5-9,10-14,15-19,20-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80-84,85+
0,E01003189,1691,109,154,163,121,101,71,96,116,133,102,142,115,88,58,55,30,17,20
1,E01003190,2418,223,239,230,133,95,142,250,190,215,193,148,140,79,37,37,40,18,9
2,E01003191,1633,122,176,84,68,69,104,144,149,109,110,90,146,85,47,48,42,17,23
3,E01003192,1740,134,137,128,90,83,127,129,141,148,123,138,105,76,60,59,15,31,16
4,E01003193,1706,117,123,106,98,75,111,128,125,148,111,142,90,78,64,103,46,25,16


...

,OA11CD,Total,0-4,5-9,10-14,15-19,20-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80-84,85+
0,E00016115,162,5,8,16,15,8,7,6,10,13,8,13,13,13,7,10,3,2,5
1,E00016136,154,10,9,21,8,5,6,6,17,8,6,19,11,5,9,8,0,0,6
2,E00016137,211,14,31,13,11,11,11,15,16,18,13,14,12,9,8,3,6,2,4
3,E00016138,161,13,8,13,9,10,4,19,11,10,12,11,17,5,5,5,3,2,4
4,E00016139,167,10,18,10,10,10,12,12,9,16,8,18,10,12,3,5,2,2,0


'=== lsoa11 ==='

,LSOA11CD,Total,0-4,5-9,10-14,15-19,20-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80-84,85+
0,E01003189,855,52,74,73,53,44,40,58,63,65,47,75,63,44,32,31,14,8,19
1,E01003190,1264,128,88,91,61,68,92,161,84,100,113,80,71,49,23,16,24,8,7
2,E01003191,875,60,114,43,28,38,63,92,73,59,54,51,76,52,21,16,12,4,19
3,E01003192,903,72,65,59,43,46,82,70,78,87,65,45,44,40,28,31,9,26,13
4,E01003193,867,58,57,51,41,39,60,74,62,62,69,59,46,45,26,60,29,17,12


'=== wd22 ==='

,WD22CD_proposed,Total,0-4,5-9,10-14,15-19,20-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80-84,85+
0,E05013714,6014,409,500,432,309,273,365,511,476,507,433,426,414,325,193,153,99,99,90
1,E05013715,8658,586,538,406,371,377,885,951,815,648,582,547,469,373,274,271,202,161,202
2,E05013716,9508,562,470,341,410,1181,1348,1185,910,635,549,515,394,307,221,185,117,90,88
3,E05013717,8954,515,656,557,489,429,536,622,668,747,611,603,698,484,401,307,201,195,235
4,E05013718,7971,649,601,424,255,272,623,802,874,748,573,467,463,375,272,197,167,105,104


Reference 5-year age groups
--

In [20]:
# Aggregate into 5-year bins
for datname_src, datname_dst in zip(
    ['pop_est_2020_all', 'pop_est_2020_male', 'pop_est_2020_female'],
    ['pop_est_2020_5ybins_all', 'pop_est_2020_5ybins_male', 'pop_est_2020_5ybins_female']):

  IPython.display.display(f"== {datname_src} ==")

  d = pd.read_csv(f"{pop_dir}/references_{datname_src}.csv")
  out = aggregate_age_bins(d,
                            fixed_columns=[d.columns[0], d.columns[1], 
                                           d.columns[2], 'Total'],
                            age_columns=[str(a) for a in range(90)] + ['90+'],
                            bin_size=5,
                            max_bin=85)
  out.to_csv(f"{pop_dir}/references_{datname_dst}.csv", index=False)
  IPython.display.display(out.head())

'== pop_est_2020_all =='

,Code,Name,Geography,Total,0-4,5-9,10-14,15-19,20-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80-84,85+
0,K02000001,United Kingdom,Country,67081234,3782330,4147413,4045114,3683680,4133158,4476630,4521975,4404100,4091543,4303967,4616017,4510851,3855818,3355381,3363906,2403759,1726223,1659369
1,K03000001,Great Britain,Country,65185724,3664594,4019539,3918097,3571812,4020834,4356811,4395393,4279748,3973599,4181028,4485434,4383348,3746798,3264688,3281844,2339225,1683051,1619881
2,K04000001,England and Wales,Country,59719724,3400788,3721636,3620016,3289692,3679079,3979607,4021324,3924082,3649233,3831104,4092321,3984004,3394229,2964255,2996014,2141015,1539755,1491570
3,E92000001,England,Country,56550138,3239447,3539458,3435579,3115871,3472522,3771493,3824652,3738209,3476303,3638639,3875351,3761782,3196813,2784300,2814128,2009992,1449189,1406410
4,E12000007,London,Region,9002488,595799,606333,550753,474456,556594,757848,822084,779934,677463,598535,569938,508722,405576,318142,280432,196419,150980,152480


'== pop_est_2020_male =='

,Code,Name,Geography,Total,0-4,5-9,10-14,15-19,20-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80-84,85+
0,K02000001,United Kingdom,Country,33145709,1941390,2125958,2073515,1893268,2132032,2280809,2263511,2179535,2032071,2126397,2269897,2216617,1888526,1624419,1606864,1114488,759183,617229
1,K03000001,Great Britain,Country,32211554,1880721,2060639,2008368,1835564,2073458,2220432,2200628,2118777,1975144,2066553,2206111,2154022,1835105,1579557,1567824,1084766,740572,603313
2,K04000001,England and Wales,Country,29546342,1744762,1907792,1856493,1691357,1900156,2031293,2014991,1944698,1815558,1897177,2016756,1960674,1664404,1435028,1431914,995560,680302,557427
3,E92000001,England,Country,27982818,1662294,1814361,1761874,1601452,1791701,1924416,1916412,1852969,1730268,1803208,1911318,1852593,1568489,1347714,1343927,934074,640018,525730
4,E12000007,London,Region,4514378,305415,310783,282709,244831,284070,381949,424018,406216,348450,303243,282573,249045,197934,151061,130224,87693,65638,58526


'== pop_est_2020_female =='

,Code,Name,Geography,Total,0-4,5-9,10-14,15-19,20-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80-84,85+
0,K02000001,United Kingdom,Country,33935525,1840940,2021455,1971599,1790412,2001126,2195821,2258464,2224565,2059472,2177570,2346120,2294234,1967292,1730962,1757042,1289271,967040,1042140
1,K03000001,Great Britain,Country,32974170,1783873,1958900,1909729,1736248,1947376,2136379,2194765,2160971,1998455,2114475,2279323,2229326,1911693,1685131,1714020,1254459,942479,1016568
2,K04000001,England and Wales,Country,30173382,1656026,1813844,1763523,1598335,1778923,1948314,2006333,1979384,1833675,1933927,2075565,2023330,1729825,1529227,1564100,1145455,859453,934143
3,E92000001,England,Country,28567320,1577153,1725097,1673705,1514419,1680821,1847077,1908240,1885240,1746035,1835431,1964033,1909189,1628324,1436586,1470201,1075918,809171,880680
4,E12000007,London,Region,4488110,290384,295550,268044,229625,272524,375899,398066,373718,329013,295292,287365,259677,207642,167081,150208,108726,85342,93954


Lewisham coarse age groups
--

In [21]:
# Quick demo: how to bin intro three age brackets, 0-18, 19-65, 66+
np.digitize([0, 17, 18, 19, 50, 65, 66, 100], [19, 66])

array([0, 0, 0, 1, 1, 1, 2, 2])

In [22]:
np.split([str(a) for a in range(90)] + ['90+'], [19, 66])

[array(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
        '13', '14', '15', '16', '17', '18'], dtype='<U3'),
 array(['19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29',
        '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40',
        '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51',
        '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62',
        '63', '64', '65'], dtype='<U3'),
 array(['66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76',
        '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87',
        '88', '89', '90+'], dtype='<U3')]

In [23]:
# Aggregates groups of columns.
# - group_columns: a dict { group_name: [list of columns] }
def aggregate_groups(d, fixed_columns, group_columns):
  out = d[fixed_columns].copy()
  for group in group_columns.keys():
    out[group] = d[group_columns[group]].sum(axis=1)
  return out

In [24]:
coarse_group_columns = dict(zip(
  # Group names
  ['0-18', '19-65', '66+'], 
  np.split(
      # Input column names, in consecutive years starting at '0'
      [str(a) for a in range(90)] + ['90+'], 
      # Split points
      [19, 66])))
coarse_group_columns

{'0-18': array(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
        '13', '14', '15', '16', '17', '18'], dtype='<U3'),
 '19-65': array(['19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29',
        '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40',
        '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51',
        '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62',
        '63', '64', '65'], dtype='<U3'),
 '66+': array(['66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76',
        '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87',
        '88', '89', '90+'], dtype='<U3')}

In [25]:
# Aggregate into coarser age groups
for datname_src, datname_dst in zip(
    ['pop_est_2020_all', 'pop_est_2020_male', 'pop_est_2020_female'],
    ['pop_est_2020_agegroups_all', 'pop_est_2020_agegroups_male', 'pop_est_2020_agegroups_female']):

  IPython.display.display(f"== {datname_src} ==")

  for level in ['oa11', 'lsoa11', 'wd22']:
    IPython.display.display(f"=== {level} ===")
    d = pd.read_csv(f"{pop_dir}/lbl_{datname_src}_{level}.csv")

    out = aggregate_groups(d,
                           fixed_columns=[d.columns[0], 'Total'],
                           group_columns=coarse_group_columns)
    out.to_csv(f"{pop_dir}/lbl_{datname_dst}_{level}.csv", index=False)
    IPython.display.display(out.head())

'== pop_est_2020_all =='

'=== oa11 ==='

,OA11CD,Total,0-18,19-65,66+
0,E00016115,326,75,202,49
1,E00016136,314,89,191,34
2,E00016137,404,145,228,31
3,E00016138,295,95,176,24
4,E00016139,352,121,209,22


'=== lsoa11 ==='

,LSOA11CD,Total,0-18,19-65,66+
0,E01003189,1691,525,1006,160
1,E01003190,2418,800,1488,130
2,E01003191,1633,440,1024,169
3,E01003192,1740,476,1091,173
4,E01003193,1706,428,1038,240


...

,OA11CD,Total,0-18,19-65,66+
0,E00016115,162,41,96,25
1,E00016136,154,46,89,19
2,E00016137,211,66,124,21
3,E00016138,161,43,101,17
4,E00016139,167,47,111,9


'=== lsoa11 ==='

,LSOA11CD,Total,0-18,19-65,66+
0,E01003189,855,243,521,91
1,E01003190,1264,356,838,70
2,E01003191,875,239,568,68
3,E01003192,903,233,568,102
4,E01003193,867,198,531,138


'=== wd22 ==='

,WD22CD_proposed,Total,0-18,19-65,66+
0,E05013714,6014,1600,3832,582
1,E05013715,8658,1837,5776,1045
2,E05013716,9508,1652,7210,646
3,E05013717,8954,2141,5563,1250
4,E05013718,7971,1880,5292,799


References coarse age groups
--

In [26]:
# Aggregate into coarser age groups
for datname_src, datname_dst in zip(
    ['pop_est_2020_all', 'pop_est_2020_male', 'pop_est_2020_female'],
    ['pop_est_2020_agegroups_all', 'pop_est_2020_agegroups_male', 'pop_est_2020_agegroups_female']):

  IPython.display.display(f"== {datname_src} ==")

  d = pd.read_csv(f"{pop_dir}/references_{datname_src}.csv")

  out = aggregate_groups(d,
                          fixed_columns=[d.columns[0], d.columns[1], 
                                         d.columns[2], 'Total'],
                          group_columns=coarse_group_columns)
  out.to_csv(f"{pop_dir}/references_{datname_dst}.csv", index=False)
  IPython.display.display(out.head())

'== pop_est_2020_all =='

,Code,Name,Geography,Total,0-18,19-65,66+
0,K02000001,United Kingdom,Country,67081234,14908442,40353352,11819440
1,K03000001,Great Britain,Country,65185724,14444998,39222149,11518577
2,K04000001,England and Wales,Country,59719724,13363513,35830708,10525503
3,E92000001,England,Country,56550138,12699899,33957363,9892876
4,E12000007,London,Region,9002488,2140450,5833055,1028983


'== pop_est_2020_male =='

,Code,Name,Geography,Total,0-18,19-65,66+
0,K02000001,United Kingdom,Country,33145709,7648834,20110437,5386438
1,K03000001,Great Britain,Country,32211554,7411092,19550660,5249802
2,K04000001,England and Wales,Country,29546342,6856652,17885385,4804305
3,E92000001,England,Country,27982818,6516281,16953452,4513085
4,E12000007,London,Region,4514378,1098302,2956572,459504


'== pop_est_2020_female =='

,Code,Name,Geography,Total,0-18,19-65,66+
0,K02000001,United Kingdom,Country,33935525,7259608,20242915,6433002
1,K03000001,Great Britain,Country,32974170,7033906,19671489,6268775
2,K04000001,England and Wales,Country,30173382,6506861,17945323,5721198
3,E92000001,England,Country,28567320,6183618,17003911,5379791
4,E12000007,London,Region,4488110,1042148,2876483,569479


Derivatives
==
Relative pop shares in Lewisham
--

In [29]:
# Lewisham
for variable in ['pop_est_2020_all', 'pop_est_2020_female', 'pop_est_2020_male',
                 'pop_est_2020_5ybins_all', 'pop_est_2020_5ybins_female', 'pop_est_2020_5ybins_male',
                 'pop_est_2020_agegroups_all', 'pop_est_2020_agegroups_female', 'pop_est_2020_agegroups_male',]:

  IPython.display.display(f"=== {variable} ===")

  for geog, geog_colname in zip(['oa11', 'wd22'], ['OA11CD', 'WD22CD_proposed']):
    IPython.display.display(f"=== {geog} ===")

    ea = pd.read_csv(f"{pop_dir}/lbl_{variable}_{geog}.csv")
    d = ea.set_index(geog_colname)
    d = (d.drop(columns='Total') * 100.).divide(d.Total, axis=0)
    d.to_csv(f"{pop_dir}/lbl_{variable}_share_{geog}.csv", index=True)
    IPython.display.display(d.head(2))

'=== pop_est_2020_all ==='

'=== oa11 ==='

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90+
OA11CD,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
E00016115,0.306748,0.613497,0.613497,0.920245,1.226994,0.920245,0.000000,1.226994,2.760736,1.226994,2.453988,0.613497,1.840491,1.226994,2.147239,2.147239,1.840491,0.613497,0.306748,1.533742,2.760736,1.226994,0.920245,0.306748,1.533742,1.533742,0.613497,0.920245,0.920245,0.920245,0.920245,0.920245,0.613497,0.920245,0.920245,0.920245,1.226994,1.226994,1.226994,0.920245,...,2.147239,1.533742,2.760736,0.920245,1.226994,2.760736,1.840491,1.840491,0.306748,1.840491,3.067485,0.920245,0.613497,1.533742,1.533742,1.226994,1.226994,0.000000,0.613497,0.613497,1.226994,2.147239,1.533742,0.613497,1.226994,0.920245,0.306748,0.306748,0.306748,1.226994,0.0,0.0,0.000000,0.000000,0.306748,0.306748,0.306748,0.0,0.306748,0.306748
E00016136,1.273885,0.955414,0.318471,1.592357,1.592357,1.592357,0.636943,1.592357,1.273885,1.273885,1.910828,1.910828,2.547771,2.866242,2.229299,1.910828,1.273885,0.955414,0.636943,1.592357,0.318471,0.955414,1.592357,0.955414,1.592357,0.636943,0.955414,0.955414,0.636943,0.318471,0.636943,1.273885,0.318471,0.636943,0.955414,3.184713,1.592357,1.910828,1.273885,1.910828,...,1.910828,1.273885,3.503185,1.592357,1.273885,1.273885,1.592357,0.636943,1.910828,0.955414,1.592357,0.636943,0.318471,0.955414,1.592357,1.592357,0.318471,0.636943,0.318471,0.636943,0.636943,0.636943,0.636943,0.955414,0.000000,0.955414,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.318471,0.955414,0.000000,0.318471,0.000000,0.0,1.273885,0.636943


'=== wd22 ==='

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90+
WD22CD_proposed,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
E05013714,1.301147,1.480911,1.429550,1.489471,1.420990,1.737716,1.549392,1.737716,2.097244,1.968841,1.720596,1.566513,1.848998,1.754837,1.669235,1.446670,1.181305,1.112823,1.070022,0.984420,0.787536,0.856018,0.881698,1.044342,0.907379,1.070022,1.027221,1.155624,1.035782,1.078582,1.129944,1.669235,1.515152,1.540832,1.626434,1.720596,1.669235,1.720596,1.42955,1.677795,...,1.617874,1.429550,1.412429,1.463790,1.378189,1.566513,1.44667,1.284027,1.189865,1.129944,1.232666,0.967300,0.933059,0.813217,0.701935,0.719055,0.573532,0.496490,0.47937,0.624893,0.436569,0.573532,0.487930,0.385208,0.428009,0.350967,0.385208,0.350967,0.248245,0.308166,0.368088,0.291046,0.239685,0.205444,0.171204,0.162643,0.128403,0.068481,0.111282,0.393768
E05013715,1.282500,1.282500,1.329136,1.270841,1.416579,1.235863,1.200886,1.299988,1.160079,1.160079,1.020170,0.961875,0.886091,0.967704,0.926898,0.996852,1.002682,0.891920,0.903579,0.792818,0.635420,0.752011,0.956045,1.026000,1.136761,1.620613,1.579806,1.935409,2.238545,2.098636,2.069488,2.471727,2.337647,1.982045,2.063659,2.121954,1.853795,1.906261,2.04034,1.987875,...,1.521511,1.160079,1.334966,1.288329,1.090125,1.171738,1.02017,0.979363,0.851113,0.950216,0.845284,0.827795,0.816136,0.763670,0.682057,0.693716,0.577125,0.495511,0.64125,0.489682,0.600443,0.664568,0.652909,0.565466,0.460534,0.402239,0.454704,0.419727,0.297307,0.378920,0.355602,0.373091,0.308966,0.378920,0.268159,0.256500,0.198205,0.151568,0.180716,0.705375


...

'=== pop_est_2020_agegroups_male ==='

'=== oa11 ==='

,0-18,19-65,66+
OA11CD,,,
E00016115,20.731707,64.634146,14.634146
E00016136,26.875000,63.750000,9.375000


'=== wd22 ==='

,0-18,19-65,66+
WD22CD_proposed,,,
E05013714,32.745236,58.997883,8.256881
E05013715,21.174670,68.973635,9.851695


Relative pop shares in reference geographies
--

In [30]:
# Lewisham
for variable in ['pop_est_2020_all', 'pop_est_2020_female', 'pop_est_2020_male',
                 'pop_est_2020_5ybins_all', 'pop_est_2020_5ybins_female', 'pop_est_2020_5ybins_male',
                 'pop_est_2020_agegroups_all', 'pop_est_2020_agegroups_female', 'pop_est_2020_agegroups_male',]:

  IPython.display.display(f"=== {variable} ===")

  ea = pd.read_csv(f"{pop_dir}/references_{variable}.csv")
  d = ea.set_index('Name')
  d = (d.drop(columns=['Code', 'Geography', 'Total']) * 100.).divide(d.Total, axis=0)
  d.to_csv(f"{pop_dir}/references_{variable}_share.csv", index=True)
  IPython.display.display(d.head(2))

'=== pop_est_2020_all ==='

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90+
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
United Kingdom,1.046339,1.088559,1.131992,1.167720,1.203823,1.202639,1.212072,1.238349,1.274574,1.255038,1.231462,1.218354,1.227962,1.187664,1.164730,1.122090,1.104173,1.077690,1.069229,1.118189,1.161174,1.207943,1.233223,1.276962,1.282120,1.280146,1.318534,1.322473,1.363428,1.388865,1.362804,1.350482,1.362066,1.328263,1.337428,1.333345,1.300611,1.309166,1.306505,1.315696,...,1.376920,1.376476,1.393406,1.390271,1.399405,1.383640,1.351560,1.318650,1.271205,1.216747,1.187414,1.159446,1.113934,1.070441,1.027408,1.024629,1.005891,0.971489,0.972549,0.985100,1.002728,1.047111,1.122533,0.857204,0.819811,0.805705,0.737799,0.648542,0.571498,0.576748,0.556472,0.523163,0.480167,0.436782,0.395039,0.345584,0.307728,0.276067,0.240649,0.908604
Great Britain,1.043095,1.085169,1.128822,1.164416,1.200272,1.199324,1.208943,1.235370,1.271217,1.251434,1.227629,1.213649,1.223450,1.183906,1.162032,1.119145,1.101123,1.074705,1.066056,1.118411,1.161816,1.208360,1.234566,1.279056,1.284476,1.282305,1.320898,1.324824,1.365030,1.390631,1.363981,1.351347,1.362826,1.327694,1.337029,1.333490,1.300168,1.309455,1.306907,1.315448,...,1.377125,1.375965,1.393699,1.390436,1.399185,1.383759,1.351771,1.318895,1.270789,1.216467,1.187438,1.159312,1.113959,1.070705,1.028004,1.025718,1.007204,0.973038,0.974322,0.987465,1.006041,1.052069,1.129964,0.859067,0.821155,0.807322,0.738680,0.649210,0.572187,0.578610,0.558003,0.525152,0.481696,0.438472,0.396542,0.347005,0.308690,0.277303,0.241823,0.913660


'=== pop_est_2020_female ==='

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90+
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
United Kingdom,1.007189,1.045768,1.089976,1.124064,1.157819,1.157513,1.167679,1.193266,1.228786,1.209508,1.188339,1.173838,1.182822,1.142007,1.122832,1.080307,1.061911,1.034294,1.024437,1.074974,1.108393,1.155149,1.179926,1.225857,1.227522,1.232257,1.270716,1.282473,1.335029,1.350093,1.325446,1.328054,1.349518,1.327989,1.324155,1.323784,1.298150,1.309065,1.30951,1.314761,...,1.385074,1.38358,1.394188,1.398900,1.407661,1.391777,1.358824,1.324014,1.278292,1.22573,1.197312,1.168784,1.124208,1.081112,1.041543,1.043196,1.024222,0.994341,0.997436,1.014648,1.031152,1.080732,1.159634,0.891423,0.857511,0.845035,0.782731,0.694797,0.619103,0.626892,0.609859,0.578205,0.538380,0.496303,0.453967,0.404384,0.365481,0.334814,0.297385,1.214910
Great Britain,1.004426,1.042625,1.086917,1.121026,1.154916,1.154361,1.164727,1.190195,1.225681,1.205747,1.184582,1.169467,1.178516,1.138628,1.120398,1.077722,1.059211,1.031219,1.021199,1.076127,1.110369,1.155984,1.181722,1.227746,1.229941,1.234594,1.272614,1.284263,1.336376,1.351100,1.326108,1.328761,1.350154,1.327648,1.323342,1.323733,1.297376,1.308918,1.30944,1.314059,...,1.384948,1.38314,1.394064,1.399247,1.407314,1.391932,1.359637,1.324033,1.277909,1.22535,1.197477,1.168748,1.124265,1.081707,1.042883,1.045309,1.025945,0.996495,0.999825,1.017448,1.034731,1.085795,1.166938,0.893157,0.858821,0.846681,0.783562,0.695602,0.619703,0.628759,0.611415,0.580285,0.539871,0.497905,0.455393,0.405845,0.366372,0.336081,0.298716,1.220516


'=== pop_est_2020_male ==='

...

,0-18,19-65,66+
Name,,,
United Kingdom,22.224460,60.155948,17.619592
Great Britain,22.159757,60.169845,17.670398


'=== pop_est_2020_agegroups_female ==='

,0-18,19-65,66+
Name,,,
United Kingdom,21.392355,59.651103,18.956542
Great Britain,21.331563,59.657268,19.011168


'=== pop_est_2020_agegroups_male ==='

,0-18,19-65,66+
Name,,,
United Kingdom,23.076393,60.672822,16.250785
Great Britain,23.007558,60.694557,16.297885


Results
==

In [31]:
!ls -lh '{pop_dir}'

total 36M
-rw------- 1 root root  13K Dec  7 11:21 lbl_pop_est_2020_5ybins_all_lsoa11.csv
-rw------- 1 root root  55K Dec  7 11:21 lbl_pop_est_2020_5ybins_all_oa11.csv
-rw------- 1 root root 286K Dec  7 11:22 lbl_pop_est_2020_5ybins_all_share_oa11.csv
-rw------- 1 root root 6.5K Dec  7 11:22 lbl_pop_est_2020_5ybins_all_share_wd22.csv
-rw------- 1 root root 1.9K Dec  7 11:21 lbl_pop_est_2020_5ybins_all_wd22.csv
-rw------- 1 root root  12K Dec  7 11:21 lbl_pop_est_2020_5ybins_female_lsoa11.csv
-rw------- 1 root root  51K Dec  7 11:21 lbl_pop_est_2020_5ybins_female_oa11.csv
-rw------- 1 root root 278K Dec  7 11:22 lbl_pop_est_2020_5ybins_female_share_oa11.csv
-rw------- 1 root root 6.4K Dec  7 11:22 lbl_pop_est_2020_5ybins_female_share_wd22.csv
-rw------- 1 root root 1.8K Dec  7 11:21 lbl_pop_est_2020_5ybins_female_wd22.csv
-rw------- 1 root root  12K Dec  7 11:21 lbl_pop_est_2020_5ybins_male_lsoa11.csv
-rw------- 1 root root  50K Dec  7 11:21 lbl_pop_est_2020_5ybins_male_oa11.csv
-rw----

In [ ]:
# !rm '{pop_dir}'/*.csv